Generar texto con modelos de Markov de segundo orden.

En realidad, en generación de texto se suelen usar más bien redes neuronales, 
porque cuanto más complejo es lo que queremos generar, más grandes son los
tensores, porque más cantidad de órdenes se necesitan.

In [75]:
import numpy as np
import string

In [3]:
def remove_punctuation(frase):
    return frase.translate(str.maketrans("","",string.punctuation))

In [4]:
def add2dict(diccionario, clave, valor):
    if clave not in diccionario:
        diccionario[clave] = []
    diccionario[clave].append(valor)

In [100]:
''' 
Ejemplos_

La casa del perro
El vino está frío
La viuda del general

initial_word = {"la":2, "el":1}
first_order = {"la":["casa", "viuda"],
               "casa":["del"],
               "del":["perro", "general"],
               "perro":["END"]...}
sec_order = {("la":"casa"):["del"]}
'''

initial_word = {}
first_order = {}
sec_order = {}

#Añadimos los tokens a los diccionarios

with open("Neruda.txt", "r", encoding="utf-8") as archivo:
    for line in archivo:
        tokens_line = remove_punctuation(line.rstrip().lower()).split()
        size = len(tokens_line)
        for i in range(size):
            token = tokens_line[i]
            #si i es igual a 0 significa que estamos al inicio de la linea
            if i == 0:
                initial_word[token] = initial_word.get(token, 0.) + 1
            else:
                #guardamos la palabra anterior de la fila
                t_1 = tokens_line[i-1]
                #si estamos al final de la linea
                if i == size - 1:
                    add2dict(sec_order, (t_1, token), 'END')
                if i == 1:
                    #la segunda palabra en la frase es la única que no tiene
                    #segundo orden (porque solo tiene una palabra antes)
                    add2dict(first_order, t_1, token)
                else:
                    #guardamos las dos palabras anteriores a la actual
                    t_2 = tokens_line[i-2]
                    add2dict(sec_order, (t_2, t_1), token)

In [101]:
#Normalizar

#Sumamos todos los valores de las palabras iniciales
inicial_total = sum(initial_word.values())
print(inicial_total)

#Para cada uno de los valores de las palabras iniciales
#lo dividimos por inicial_total, para calcular su probabilidad a priori

for clave, valor in initial_word.items():
    initial_word[clave] = valor/inicial_total

503.0


In [102]:
# 'para': ["sobrevivirme", "que", "tus", "que", "tus", "mi"...]
# A cada palabra única de la lista le vamos a dar un valor de probabilidad


def list2prob_dict(word_list):
    diccionario = {} #creamos un diccionario vacio
    n = len(word_list)

    #Ciclo para contar la ocurrencia de cada elemento en la lista
    for palabra in word_list:
        diccionario[palabra] = diccionario.get(palabra, 0.) + 1

    #Ciclo para convertir los conteos en probabilidades relativas
    for clave, valor in diccionario.items():
        diccionario[clave] = valor/n

    #Devolver diccionario de probabilidades
    return diccionario




In [103]:
for t_1, lista in first_order.items():
    #reemplazamos la lista con diccionario de probabilidades:
    first_order[t_1] = list2prob_dict(lista)

In [104]:
for item, lista in sec_order.items():
    #reemplazamos la lista con diccionario de probabilidades:
    sec_order[item] = list2prob_dict(lista)

In [112]:
first_order

{'veinte': {'poemas': 1.0},
 'y': {'una': 0.044444444444444446,
  'hace': 0.022222222222222223,
  'en': 0.044444444444444446,
  'la': 0.06666666666666667,
  'llena': 0.022222222222222223,
  'a': 0.022222222222222223,
  'soltaré': 0.022222222222222223,
  'tu': 0.06666666666666667,
  'eres': 0.022222222222222223,
  'desvía': 0.022222222222222223,
  'sustancia': 0.022222222222222223,
  'las': 0.044444444444444446,
  'están': 0.022222222222222223,
  'caían': 0.022222222222222223,
  'te': 0.044444444444444446,
  'embriagadoras': 0.022222222222222223,
  'como': 0.044444444444444446,
  'entristeces': 0.022222222222222223,
  'suelta': 0.022222222222222223,
  'tienes': 0.022222222222222223,
  'sobre': 0.022222222222222223,
  'me': 0.044444444444444446,
  'parece': 0.022222222222222223,
  'estás': 0.022222222222222223,
  'estoy': 0.022222222222222223,
  'viven': 0.022222222222222223,
  'mis': 0.022222222222222223,
  'mi': 0.022222222222222223,
  'él': 0.022222222222222223,
  'amo': 0.02222222222

In [115]:
initial_word

{'veinte': 0.0019880715705765406,
 'y': 0.08946322067594434,
 'pablo': 0.0019880715705765406,
 'poema': 0.039761431411530816,
 'cuerpo': 0.005964214711729622,
 'te': 0.013916500994035786,
 'mi': 0.02982107355864811,
 'fui': 0.0019880715705765406,
 'para': 0.015904572564612324,
 'como': 0.027833001988071572,
 'pero': 0.007952286282306162,
 'ah': 0.02186878727634195,
 'oscuros': 0.0019880715705765406,
 'en': 0.033797216699801194,
 'absorta': 0.0019880715705765406,
 'contra': 0.0019880715705765406,
 'que': 0.011928429423459244,
 'muda': 0.0019880715705765406,
 'sola': 0.0019880715705765406,
 'pura': 0.0019880715705765406,
 'del': 0.0019880715705765406,
 'de': 0.02982107355864811,
 'crecen': 0.0019880715705765406,
 'oh': 0.019880715705765408,
 'círculo': 0.0019880715705765406,
 'erguida': 0.0019880715705765406,
 'lento': 0.0019880715705765406,
 'crepúsculo': 0.0019880715705765406,
 'caracola': 0.0019880715705765406,
 'márcame': 0.0019880715705765406,
 'donde': 0.003976143141153081,
 'así':

In [119]:
def palabra_ejemplo(diccionario, imprimir):
    #Generamos un nro aleatorio entre 0 y 1
    p0 = np.random.random()
    if (imprimir == 1):
        print(f"p0: {p0}")
    
    #Inicializamos una variable para realizar la suma acumulativa de probs
    cumulative = 0
    if(imprimir == 1):
        print(f"Probabilidad acumulada inicial: {cumulative}")

    #Recorremos cada una de las claves del diccionario y sumamos la prob acumulada
    for clave, valor in diccionario.items():
        cumulative += valor
        if (imprimir == 1):
            print(f"item: {clave}")
    #Comprueba si el nro aleatorio es menor que la acumulación de probs
        if p0 < cumulative:
            return clave

In [120]:
palabra_ejemplo(initial_word,1)

p0: 0.7648812001915698
Probabilidad acumulada inicial: 0
item: veinte
item: y
item: pablo
item: poema
item: cuerpo
item: te
item: mi
item: fui
item: para
item: como
item: pero
item: ah
item: oscuros
item: en
item: absorta
item: contra
item: que
item: muda
item: sola
item: pura
item: del
item: de
item: crecen
item: oh
item: círculo
item: erguida
item: lento
item: crepúsculo
item: caracola
item: márcame
item: donde
item: así
item: doblarse
item: es
item: el
item: innumerable
item: latiendo
item: zumbando
item: viento
item: se
item: combatido
item: mis
item: collar
item: más
item: van
item: ellas
item: eres
item: todo
item: antes
item: ahora
item: huracanes
item: escuchas
item: llanto
item: ámame
item: sígueme
item: voy
item: eras
item: apegada
item: las
item: hoguera
item: dulce
item: siento
item: boina
item: hacia
item: cielo
item: tu
item: hojas
item: inclinado
item: a
item: allí
item: náufrago
item: hago
item: solo
item: los
item: galopa
item: desparramando
item: abeja
item: soy
item:

'con'

In [121]:
def generador (tamaño):
    #tamaño es la cantidad de lineas que queremos que tenga el texto
    for linea in range(tamaño):
        oracion = []
        #Palabra inicial
        pal0 = palabra_ejemplo(initial_word, 0)
        oracion.append(pal0)

        #Segunda palabra
        pal1 = palabra_ejemplo(first_order[pal0], 0)
        oracion.append(pal1)

        #Segundo orden hasta el fin
        while True:
            pal2 = palabra_ejemplo(sec_order[(pal0,pal1)], 0)
            if pal2 == 'END':
                break
            oracion.append(pal2)
            pal0 = pal1
            pal1 = pal2

        print(' '.join(oracion))

In [125]:
generador(5)

como un volante loco
pero tú clara niña pregunta de humo entre las estrellas más grandes
más que tú me oigas
allí se estira y arde en la profunda soledad
poema 19
